In [1]:
!pip install requests

## Install Llama.cpp (only need to do this once)
The instructions below are for Macs with an M1 chip.
For other operating systems, comment out those cells and get instructions [here](https://github.com/TrelisResearch/llamacpp-install-basics/blob/main/instructions.md).

In [2]:
# Set the SYSTEM PROMPT
DEFAULT_SYSTEM_PROMPT = 'You are a helpful pair-coding assistant.'
SYSTEM_PROMPT = DEFAULT_SYSTEM_PROMPT

print(SYSTEM_PROMPT)

You are a helpful pair-coding assistant.


In [3]:
# Download the model file
model_name = 'TheBloke/CodeLlama-7B-Instruct-GGUF/resolve/main/codellama-7b-instruct.Q2_K.gguf'

In [4]:
pure_name = model_name.split('/')[-1]
print(pure_name)

codellama-7b-instruct.Q2_K.gguf


In [5]:
parts = model_name.split('/')
model_path = f"{parts[0]}/{parts[1]}"

print(model_path)

TheBloke/CodeLlama-7B-Instruct-GGUF


In [6]:
import os
from IPython.display import display, Markdown

def process_os_choice():
    """Process the choice of OS made by the user."""
    
    if not os.path.exists('llama.cpp'):
        print("Cloning llama.cpp...")
        !git clone https://github.com/ggerganov/llama.cpp
        %cd llama.cpp
        print("Compiling for Mac with M1 chip...")
        !LLAMA_METAL=1 make
        print("Compilation completed!")
        
        
        %cd ../

    else:
        print("llama.cpp has already been cloned into this directory!")
        
process_os_choice()

llama.cpp has already been cloned into this directory!


In [7]:
import os

%cd llama.cpp

if not os.path.exists(pure_name):
    !wget https://huggingface.co/{model_name}
else:
    print(f"{pure_name} already exists!")

%cd ../

/Users/niccolominetti/Desktop/GTL-machine/llama.cpp
codellama-7b-instruct.Q2_K.gguf already exists!
/Users/niccolominetti/Desktop/GTL-machine


/Users/niccolominetti/Desktop/GTL-machine/llamaEnv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
!pip install transformers -q #for the tokenizer so we can shorten text.
!pip install torch -q

In [10]:
#!pip install PyPDF2 -q
#!pip install ipywidgets -q
#!pip install jupyterlab_widgets -q

In [11]:
# Set the default value for context_length to High Speed
context_length = 4096
max_doc_length = int(0.75 * context_length)
max_doc_tokens = max_doc_length
n_predict = int(0.2 * context_length)

def initialize_preference():
    global context_length, max_doc_length, max_doc_tokens, n_predict
    
    context_length = 4096
        
    # If a valid choice was made, update other related values
    if context_length:
        max_doc_length = int(0.75 * context_length) # Makes inference faster
        max_doc_tokens = int(0.75 * max_doc_length)
        n_predict = int(0.2 * context_length)

initialize_preference()

In [12]:
import os
import subprocess
import threading

def start_server():
    # Change directory using Python's os module
    os.chdir('llama.cpp')
    
    base_command = f"./server -m {pure_name} -c {context_length} --port 8081"
    command = f"{base_command} -ngl 48"
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()
    
    print(out.decode())
    if err:
        print(err.decode())

    # Change back to the parent directory using Python
    os.chdir('../')

thread = threading.Thread(target=start_server)
thread.start()

## Set up the User Interface

In [13]:
def build_coding_task_prompt(task_description):
    """
    Builds a detailed and structured prompt for a code-generating model based on a provided coding task description,
    emphasizing high-quality code and well-written documentation.

    :param task_description: A string describing the coding task.
    :return: A string formatted as a detailed prompt for the code-generating model.
    """

    # Introduction specifying the task
    prompt_intro = "Create a Python solution for the following task:"

    # Instruction for high-quality code
    quality_instruction = "\nEnsure the code is of high quality, following best practices for efficiency, readability, and maintainability."

    # Instruction for well-written documentation
    documentation_instruction = "\nInclude comprehensive documentation and comments to explain the code's logic and any important decisions."
    
    # Instruction for formatting
    format_instruction = """\nAlways start code with '\\begin{code}' and end with '\end{code}'."""
    # Elicit llm response
    output_instruction = "\n\\begin{code}"#"\nCode:"

    # Combining all parts to form the full prompt
    prompt = f"{prompt_intro} {task_description}{quality_instruction}{documentation_instruction}{format_instruction}{output_instruction}"

    # Optionally, specify any additional constraints or requirements
    # Example: prompt += "\nThe solution should handle edge cases and include error checking."

    return prompt


In [14]:
import requests
import json
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer_path = 'Trelis/Llama-2-7b-chat-hf-function-calling-GGML'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Define the necessary constants
context_length = 1024  # Adjust this according to your model's maximum context length

def generate_simple_response(input_string, n_predict):
    url = 'http://127.0.0.1:8081/completion'  # Endpoint for the completion

    # Tokenize the input string
    input_ids = tokenizer.encode(input_string, truncation=True, max_length=context_length)
    
    # Check if the tokenized input exceeds the context length
    if len(input_ids) > context_length:
        print("\n\n **The language model's input limit has been reached.**")
        return

    # Prepare parameters for the POST request
    params = {
        "prompt": input_string,
        "n_predict": n_predict,
        "n_ctx": context_length
    }
    
    # Send the POST request to the model server
    response = requests.post(url, json=params)

    # Check response status and return the content
    if response.status_code != 200:
        return f"Error: {response.status_code}, {response.text}"

    # Parse and return the response content
    new_assistant_response = json.loads(response.text)['content']
    return new_assistant_response


In [50]:
def build_test_case_prompt(task_description, generated_code):
    """
    Builds a prompt for generating test cases for the provided code based on the coding task description.

    :param task_description: A string describing the coding task.
    :param generated_code: The Python code generated from the first prompt.
    :return: A string formatted as a prompt for creating test cases.
    """

    # Introduction specifying the task for test case generation
    prompt_intro = "Create comprehensive test cases for the Python solution to the following task:"

    # Include the task description
    task_info = f"\nTask Description: {task_description}"

    # Include the generated code with formatting
    code_intro = "\nGenerated Code:\n\\begin{code}"
    code_outro = "\\end{code}"

    # Start output
    output_start = "\n\\begin{code}"

    # Combining all parts to form the full prompt for test case generation
    prompt = f"""{prompt_intro}{task_info}{code_intro}{generated_code}{code_outro}
Write test cases to thoroughly test the functionality of the above code, ensuring it handles a variety of inputs, including edge cases.{output_start}
"""

    return prompt


# Task driven Code Generation

In [16]:
# Start run

In [17]:
# Example usage
task_description = "Write a function to calculate the factorial of a number, handling both positive integers and edge cases like zero."
prompt = build_coding_task_prompt(task_description)
print(prompt)

Create a Python solution for the following task: Write a function to calculate the factorial of a number, handling both positive integers and edge cases like zero.
Ensure the code is of high quality, following best practices for efficiency, readability, and maintainability.
Include comprehensive documentation and comments to explain the code's logic and any important decisions.
Always start code with '\begin{code}' and end with '\end{code}'.
\begin{code}


In [18]:
response = generate_simple_response(prompt, n_predict=n_predict)

In [19]:
code = response.split("\end{code}")[0]
print(code)


def calculate_factorial(n):
    if n == 0:
        return 1
    else:
        result = 1
        while n > 0:
            result *= n
            n -= 1
        return result



# Unit Test Generation

In [51]:
test_case_prompt = build_test_case_prompt(task_description, code)
print(test_case_prompt)

Create comprehensive test cases for the Python solution to the following task:
Task Description: Write a function to calculate the factorial of a number, handling both positive integers and edge cases like zero.
Generated Code:
\begin{code}
def calculate_factorial(n):
    if n == 0:
        return 1
    else:
        result = 1
        while n > 0:
            result *= n
            n -= 1
        return result
\end{code}
Write test cases to thoroughly test the functionality of the above code, ensuring it handles a variety of inputs, including edge cases.
\begin{code}



In [52]:
test_response = generate_simple_response(test_case_prompt, n_predict=n_predict)
test = test_response.split("\end{code}")[0]
print(test)

import unittest

class TestCalculateFactorial(unittest.TestCase):
    def test_calculate_factorial_0(self):
        self.assertEqual(calculate_factorial(0), 1)



In [53]:
print(code + test)


def calculate_factorial(n):
    if n == 0:
        return 1
    else:
        result = 1
        while n > 0:
            result *= n
            n -= 1
        return result
import unittest

class TestCalculateFactorial(unittest.TestCase):
    def test_calculate_factorial_0(self):
        self.assertEqual(calculate_factorial(0), 1)



In [54]:
def calculate_factorial(n):
    if n == 0:
        return 1
    else:
        result = 1
        while n > 0:
            result *= n
            n -= 1
        return result
import unittest

class TestCalculateFactorial(unittest.TestCase):
    def test_calculate_factorial_0(self):
        self.assertEqual(calculate_factorial(0), 1)